In [1]:
import nltk            # natural language tool kit
import numpy as np     # support for large data structures
import pandas as pd    # data structure support
import re
import time

#from importlib import reload 
import part2_helpers

# POS tagging functions
import pos3

from nltk.tree import Tree

# Load the Penn Treebank Corpus which will serve as our training set.
corpus = nltk.corpus.ptb

In [2]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
use_full_ptb = True
if use_full_ptb:
    part2_helpers.verify_ptb_install()
    corpus = nltk.corpus.ptb  # Throws errors, for some reason
    # This configures the corpus to use the WSJ section only.
    # The Brown section has some mis-bracketed trees that will cause the 
    # corpus reader to throw (many) errors.
    if not hasattr(corpus, '_parsed_sents'):
        print("Monkey-patching corpus reader...")
        corpus._parsed_sents = corpus.parsed_sents
        corpus.parsed_sents = lambda: corpus._parsed_sents(categories=['news'])

[nltk_data] Downloading package ptb to
[nltk_data]     /Users/brianschneider/nltk_data...
[nltk_data]   Package ptb is already up-to-date!


ValueError: In category mapping file allcats.txt: WSJ/00/WSJ_0001.MRG not found

In [ ]:
train = pd.read_csv('/Users/brianschneider/Desktop/quora_project/w266_Quora_Project/apope/train_split.csv')
train_lite = pd.read_csv('/Users/brianschneider/Desktop/quora_project/w266_Quora_Project/apope/train_lite.csv')
dev = pd.read_csv('/Users/brianschneider/Desktop/quora_project/w266_Quora_Project/apope/dev_split.csv')
del train["Unnamed: 0"]
del dev["Unnamed: 0"]
del train_lite["Unnamed: 0"]
del train_lite["Unnamed: 0.1"]

In [ ]:
reload(pos3)
hmm = pos3.HMM()
for sentence in corpus.tagged_sents():
    hmm.update_counts(sentence)
hmm.compute_logprobs()

In [ ]:
def tags_as_string(p_sent):
    p = re.compile("[^A-Za-z ]")
    s = p.sub("", p_sent).split()+["?"]
    r = " ".join(hmm.viterbi(s))
    return r

In [ ]:
def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)

In [ ]:
t0 = time.time()
nrows = train_lite.shape[0]
i = 300000
while i < nrows:
    x = train_lite[i:min(i+1000,nrows)]
    f = "Data/train_lite"+str(i)+".df.pkl.gz"
    print("Starting ", f, i, min(i+1000, nrows), pretty_timedelta(since=t0))
    x["question1_pos"] = x["question1"].apply(tags_as_string)
    x["question2_pos"] = x["question2"].apply(tags_as_string)
    x.to_pickle(f)
    print("Finished ", f, i, min(i+1000, nrows), pretty_timedelta(since=t0))
    i += 1000

In [ ]:
y = pd.read_pickle("Data/train_lite8000.df.pkl.gz")
y.head()